## Train a neural network to classify images of hand-drawn numbers (classification)

In [ ]:
# Imports

import numpy as np
import matplotlib.pyplot as plt

from neuralnetwork.neural_network_module import NeuralNetwork, Layer
from neuralnetwork.neural_network_module import train_nn_classification, one_hot_to_predictions, prediction_accuracy, confusion_matrix

np.random.seed(101) # set random seed

plt.rcParams.update({'font.size': 12}) # plot parameters


### Load, transform, and plot data

In [ ]:
# Load data from sci-kit learn mnist dataset

# from sklearn.datasets import fetch_openml

# mnist = fetch_openml("mnist_784", version=1)
# mnist.keys()

# X = np.asarray(mnist["data"])
# Y = np.asarray(mnist["target"], dtype=int)


In [ ]:
# Function to save MNIST data to CSV

# # save MNIST data to csv
# import os as os
# if not os.path.exists("./data"):
#     os.mkdir("./data")
# np.savetxt("./data/mnist_data.csv", X, fmt = '%d', delimiter=",")
# np.savetxt("./data/mnist_target.csv", Y, fmt = '%d', delimiter=",")

In [ ]:
# Load MNIST data from CSV, normalize data, and plot

# load data from CSV in ./data
X = np.loadtxt("data/mnist_data.csv", delimiter=",")
X = X/255
Y = np.loadtxt("data/mnist_target.csv", delimiter=",", dtype=int)
Y = Y.reshape(-1,1)

# plot several data points
plt.figure(figsize=(24,12))
plt.suptitle("Training data and labels")
for kp in range(18):
    plt.subplot(3,6,kp+1)
    plt.imshow(X[kp,:].reshape(28,28), cmap=plt.get_cmap('gray'))
    plt.xlabel(f"Label: {Y[kp,0]}")
    plt.xticks([])
    plt.yticks([])
plt.show()


### Split data into training and test sets

In [ ]:
# Split data into training and set sets

print(f"Size of data: {X.shape}")

# training set
X_train = X[0:60000,:]
Y_train = Y[0:60000]

# test set
Y_test = Y[60000:]
X_test = X[60000:,:]

### Create and train neural network and analyze its performance

In [ ]:
# Create and train neural network

nn = NeuralNetwork()

# layer_list = []
# layer_list.append(Layer(784, None))
# layer_list.append(Layer(28, "relu"))
# layer_list.append(Layer(10, "softmax"))
# nn._set_layers(layer_list)

nn._add_layers([Layer(784, None)])
nn._add_layers([Layer(148, "relu")])
nn._add_layers([Layer(28, "relu")])
nn._add_layers([Layer(10, "softmax")])

NN_list, train_accuracy, test_accuracy, epochs = train_nn_classification(nn, X_train, Y_train, 0.10, 100, 10, X_test, Y_test)


In [ ]:
# Plot train and test accuracy accross epochs

plt.figure(figsize=(12,6))
plt.plot(epochs, train_accuracy, 'b.', label="Train accuracy")
if len(test_accuracy) == len(epochs):
    plt.plot(epochs, test_accuracy, 'g.', label="Test accuracy")
plt.xticks(epochs)
plt.yticks(np.arange(0,1.1,0.1))
plt.ylim([0, 1])
plt.title("Accuracy of NN on training and test data sets across training epochs")
plt.xlabel("Iterations")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

In [ ]:
# Plot NN results on test data

plt.figure(figsize=(24,12))
plt.suptitle("NN predictions on test data set")
for kp in range(18):
    plt.subplot(3,6,kp+1)
    plt.imshow(X_test[kp,:].reshape(28,28), cmap=plt.get_cmap('gray'))
    Y_pred = one_hot_to_predictions(nn._compute_output(X_test[[kp],:]))[0][0]
    if Y_pred == Y_test[kp,0]:
        plt.xlabel(f"Prediction: {Y_pred} | Label: {Y_test[kp,0]}", color="green")
    else:
        plt.xlabel(f"Prediction: {Y_pred} | Label: {Y_test[kp,0]}", color="red")
    plt.xticks([])
    plt.yticks([])
plt.show()

In [ ]:
# Plot confusion matrix for NN predictions

CM_train = confusion_matrix(NN_list[-1], X_train, Y_train)
CM_test = confusion_matrix(NN_list[-1], X_test, Y_test)

plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
plt.imshow(CM_train, cmap="Blues")
plt.title("Training CM")
plt.xlabel("Predictions")
plt.ylabel("Values")
plt.subplot(1,2,2)
plt.title("Test CM")
plt.imshow(CM_test, cmap="Blues")
plt.xlabel("Predictions")
plt.ylabel("Values")
plt.show()


In [ ]:
# Plot incorrect predictions

Y_pred = one_hot_to_predictions(nn._compute_output(X_test))
idx = np.where(Y_pred[:,0] != Y_test[:,0])[0]

plt.figure(figsize=(24,12))
plt.suptitle("Incorrect predictions on test data set")
for kp in range(18):
    plt.subplot(3,6,kp+1)
    plt.imshow(X_test[idx[kp],:].reshape(28,28), cmap=plt.get_cmap('gray'))
    plt.xlabel(f"Prediction: {Y_pred[idx[kp],0]} | Label: {Y_test[idx[kp],0]}", color="red")
    plt.xticks([])
    plt.yticks([])

plt.show()